In [0]:
#Install any mandatory libraries to run it on colab
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
#import some tools
from tqdm import tqdm
from google.colab import drive
import numpy as np
import pandas as pd 
from pandas import DataFrame
import spacy
import operator

In [0]:
#Setting up to use the files from google drives
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Read the file I want to use
df_train = pd.read_csv("gdrive/My Drive/5500/job-salary-prediction/Train_rev1.csv")

In [0]:
# Build a function that extract X and Y
def Generate_X_Y(df):
    df_x = df[['Title', "FullDescription", "LocationNormalized","ContractType",
                       "ContractTime","Company","Category"]]
    df_y = df["SalaryNormalized"]
    return df_x, df_y

In [0]:
# Extract X and Y from the file we want to use
df_train_x, df_train_y = Generate_X_Y(df_train)

In [0]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

In [0]:
# Use NLP to process the unstructured features
def token_remove_stopwords(text):
    # Process a paragraph of text, e.g. job title&job description
    doc = nlp(str(text))
    
    # Remove stop words and punctuation, make words be lower cases, save tokens in a list
    tokenized = []
    for token in doc:
        if token.is_stop is False and token.is_punct is False:
            tokenized.append((token.lemma_).lower())
    return tokenized

In [0]:
# Process Titles
train_New_Title = []
for i in tqdm(df_train_x["Title"]):
    train_New_Title.append(token_remove_stopwords(i))

100%|██████████| 244768/244768 [25:43<00:00, 158.63it/s]


In [0]:
# Process Descriptions
train_New_Description = []
for i in tqdm(df_train_x["FullDescription"]):
    train_New_Description.append(token_remove_stopwords(i))

100%|██████████| 244768/244768 [2:31:44<00:00, 26.88it/s]


In [0]:
len(train_New_Description)

244768

In [0]:
# Insert Title token and Description token into the original dataset
df_train_x.insert(1, "Token Title", train_New_Title) 
df_train_x.insert(3, "Token Description", train_New_Description) 

In [0]:
df_train_x

,Title,Token Title,FullDescription,Token Description,LocationNormalized,ContractType,ContractTime,Company,Category
0,Engineering Systems Analyst,"[engineering, systems, analyst]",Engineering Systems Analyst Dorking Surrey Sal...,"[engineering, systems, analyst, dorking, surre...",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs
1,Stress Engineer Glasgow,"[stress, engineer, glasgow]",Stress Engineer Glasgow Salary **** to **** We...,"[stress, engineer, glasgow, salary, currently,...",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs
2,Modelling and simulation analyst,"[modelling, simulation, analyst]",Mathematical Modeller / Simulation Analyst / O...,"[mathematical, modeller, simulation, analyst, ...",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs
3,Engineering Systems Analyst / Mathematical Mod...,"[engineering, systems, analyst, mathematical, ...",Engineering Systems Analyst / Mathematical Mod...,"[engineering, systems, analyst, mathematical, ...",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs
4,"Pioneer, Miser Engineering Systems Analyst","[pioneer, miser, engineering, systems, analyst]","Pioneer, Miser Engineering Systems Analyst Do...","[pioneer, miser, , engineering, systems, anal...",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs
...,...,...,...,...,...,...,...,...,...
244763,TEACHER OF SCIENCE,"[teacher, science]",Position: Qualified Teacher Subject/Specialism...,"[position, qualified, teacher, subject, specia...",Swindon,NaN,contract,NaN,Teaching Jobs
244764,TEACHER OF BUSINESS STUDIES AND ICT,"[teacher, business, studies, ict]",Position: Qualified Teacher or NQT Subject/Spe...,"[position, qualified, teacher, nqt, subject, s...",Swindon,NaN,contract,NaN,Teaching Jobs
244765,ENGLISH TEACHER,"[english, teacher]",Position: Qualified Teacher Subject/Specialism...,"[position, qualified, teacher, subject, specia...",Swindon,NaN,contract,NaN,Teaching Jobs
244766,SUPPLY TEACHERS,"[supply, teacher]",Position: Qualified Teacher Subject/Specialism...,"[position, qualified, teacher, subject, specia...",Wiltshire,NaN,contract,NaN,Teaching Jobs


In [0]:
# Save it on Google Drive
with open('/content/gdrive/My Drive/Train_X_modified.csv', 'w') as f:
  df_train_x.to_csv(f)

!cat /content/gdrive/My\ Drive/Train_X_modified.csv